In [4]:
from collections import defaultdict
import pandas as pd
import csv
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
f = open('data/interactions_train.csv', 'r', encoding='utf-8-sig')
f1 = open('data/interactions_test.csv', 'r', encoding='utf-8-sig')
f2 = open('data/interactions_validation.csv', 'r', encoding='utf-8-sig')
header = f.readline().strip().split(',')
header = f1.readline().strip().split(',')
header = f2.readline().strip().split(',')
# del header[-1]
dataset = []
# Print the header and fields
print("Header:", header)
for line in f:
    fields = line.strip().split(',')
    # del fields[-1]
    d = dict(zip(header, fields))
    dataset.append(d)
for line in f1:
    fields = line.strip().split(',')
    # del fields[-1]
    d = dict(zip(header, fields))
    dataset.append(d)
for line in f2:
    fields = line.strip().split(',')
    # del fields[-1]
    d = dict(zip(header, fields))
    dataset.append(d)
f.close()
f1.close()
f2.close()

df = pd.DataFrame(dataset)

item_id_label = dict(zip(df['recipe_id'], range(len(df['recipe_id']))))
item_id_label_reverse = dict(zip(range(len(df['recipe_id'])), df['recipe_id']))
user_id_label = dict(zip(df['user_id'], range(len(df['user_id']))))
user_id_label_reverse = dict(zip(range(len(df['user_id'])), df['user_id']))

ratings = df['rating'].tolist()
ratings = [float(i) for i in ratings]
user_id = df['user_id'].tolist()
user_id = [int(user_id_label[i]) for i in user_id]
recipe_id = df['recipe_id'].tolist()
recipe_id = [int(item_id_label[i]) for i in recipe_id]
# print(df)
# print(dataset)
# print(len(recipe_id), len(user_id))
# matrix = sparse.coo_matrix((ratings, (recipe_id, user_id)))
matrix = sparse.csr_matrix((ratings,(recipe_id, user_id)))
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
for d in dataset:
    user = int(d['user_id'])
    recipe = int(d['recipe_id'])
    reviewsPerUser[user].append(d)
    reviewsPerItem[recipe].append(d)
# print(reviewsPerItem)
# print(reviewsPerUser)

ratingMean = sum([i for i in ratings]) / len(dataset)
# print(ratingMean)
labels = [float(d['rating']) for d in dataset]

Header: ['user_id', 'recipe_id', 'date', 'rating', 'u', 'i']


In [46]:
print(matrix.shape)
print(len(recipe_id))
print(len(user_id))
print(len(ratings))

(718379, 718379)
718379
718379
718379


In [163]:
def Cosine(s1, s2):
    dot_product = s1.dot(s2.T)
    # print(s1, s2)
    norm_s1 = np.linalg.norm(s1)
    norm_s2 = np.linalg.norm(s2)

    if norm_s1 == 0 or norm_s2 == 0:
        return 0.1

    return sum(dot_product / (norm_s1 * norm_s2))[0] + 0.1

def predictRatingCosine(user, item):
    if str(item) not in list(item_id_label.keys()):
        return [], []
    item=item_id_label[str(item)]
    user=int(user)
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        # print(d)
        i2 = d['recipe_id']
        item2 = item_id_label[str(i2)]
        if item2 == item:
            ratings.append(0)
        else:
            ratings.append(float(d['rating']))
        similarities.append(Cosine(matrix[[item2]].toarray().astype(float), matrix[[item]].toarray().astype(float)))
    return similarities, ratings

def predict_rating(sims, ratings):
    if sum(sims) == 0:
        return ratingMean
    print(sims, ratings)
    num = sum([sims[i] * ratings[i] for i in range(len(sims))])
    return num/sum(sims)

def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [ ]:
# cfPredictions = predictRatingCosine(38094,38094)
ammount = 1000
user_a = 5
# pred_ratings = []
# always_mean = []
cfPredictions = [predictRatingCosine(2046, recipe_id[i]) for i in range(len(recipe_id[:ammount]))]
# print(cfPredictions)
pred_ratings = [predict_rating(i[0], i[1]) for i in cfPredictions]
always_mean = [ratingMean for i in cfPredictions] 
print(MSE(pred_ratings, labels[:ammount]))
print(MSE(always_mean, labels[:ammount]))

In [ ]:
item_id_label